In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#install and import pingouin package 
!pip install pingouin
import pingouin as pg
%matplotlib inline

import seaborn as sns
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
sns.set()

### Import csv dataset from websites 

In [ ]:
df = pd.read_csv("/kaggle/input/human-resources-data-set/HRDataset_v14.csv")

In [ ]:
df.head()

In [ ]:
df.info()

### Data Label
1. Data checking on the attributes label
<br>
1.1.1 Married ID = 0 includes divorced, seperated, single and widowed
<br>
1.1.2 Married ID = 1 referring to solely married.

In [ ]:
pd.crosstab(df.MarriedID, df.MaritalStatusID)

In [ ]:
pd.crosstab(df.MarriedID, df.MaritalDesc)

### Data Label
1. Data checking on the attributes label
<br>
1.2 Both Sex & GenderID are binary

In [ ]:
pd.crosstab(df.GenderID, df.Sex)

### Data Label
1. Data checking on the attributes label
<br>
1.3.1 Termd = 0 includes active, future start and leave of absence.
<br>
1.3.2 Termd = 1 includes terminated for cause and voluntarily terminated

In [ ]:
pd.crosstab(df.Termd, df.EmploymentStatus)

In [ ]:
pd.crosstab(df.Termd, df.EmpStatusID)

### Data Label
1. Data checking on the attributes label
<br>
1.4 The perfromance score (PerScoreID) is a scalar attribute instead of numeric

In [ ]:
pd.crosstab(df.PerfScoreID, df.PerformanceScore)

### Data Label
1. Data checking on the attributes label
<br>
1.5 Each deparment has their respective DeptID with producttion team is the highest number of employee

In [ ]:
pd.crosstab(df.DeptID, df.Department)

### Exploratory data analysis
1. Around 40% of the employee are married.
2. Higher portion of female employee (57%) as compared with male employee (43%)
3. Two third of the employee performance score is fully meets (3).
4. Churn rate for this company is around one third (33%).
5. Employee satisfaction on average sit on 3.89 out of 5
6. Most of employees (75th pecentile) is not involve in any special project for the past 6 months and did not have any days late for the last 30 days.


In [ ]:
df.describe()

### Objective
1. Determine whether there is relationship between who a person works for and their performance score?
<br>
1.1 Method used is Chi-Square Test for Independence
<br>
>> Null hypothesis: Assumes that there is no association between the two variables.
<br>
>> Alternative hypothesis: Assumes that there is an association between the two variables.
<br>
1.1.1 Create a table between Manager & Performance score 

In [ ]:
# chi-squared test with similar proportions
from scipy.stats import chi2_contingency
from scipy.stats import chi2
table = pd.crosstab(df['ManagerID'],df['PerformanceScore'])
stat, p, dof, expected = chi2_contingency(table)
print(table)

### Objective
1. Determine whether there is relationship between who a person works for and their performance score?
<br>
1.1.2 Check the degree of freedom and expected table

In [ ]:
print('dof=%d' % dof)
print(expected)

### Objective
1. Determine whether there is relationship between who a person works for and their performance score?
<br>
1.1.3 Results shown that we are unable to conclude that there is a relationship between performance score with manager

In [ ]:
# interpret test-statistic
prob = 0.95
critical = chi2.ppf(prob, dof)
print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
if abs(stat) >= critical:
	print('Dependent (reject H0)')
else:
	print('Independent (fail to reject H0)')
    
# interpret p-value
alpha = 1.0 - prob
print('\nsignificance=%.3f, p=%.3f' % (alpha, p))
if p <= alpha:
	print('Dependent (reject H0)')
else:
	print('Independent (fail to reject H0)')

### Objective
1. Determine whether there is relationship between who a person works for and their performance score?
<br>
1.1.4 Results shown that we are unable to conclude that there is a relationship between performance score with deparment as well

In [ ]:
table = pd.crosstab(df['Department'],df['PerformanceScore'])

stat, p, dof, expected = chi2_contingency(table)

# interpret test-statistic
prob = 0.95
critical = chi2.ppf(prob, dof)
print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
if abs(stat) >= critical:
	print('Dependent (reject H0)')
else:
	print('Independent (fail to reject H0)')
    
# interpret p-value
alpha = 1.0 - prob
print('\nsignificance=%.3f, p=%.3f' % (alpha, p))
if p <= alpha:
	print('Dependent (reject H0)')
else:
	print('Independent (fail to reject H0)')

### Objective
2. What is the overall diversity profile of the organization?
<br>
2.1 Majority of the employee are white and US citizens
<br>
2.2 Around 40% of the employee are married.
<br>
2.3 Higher portion of female employee (57%) as compared with male employee (43%)

In [ ]:
fig, ax = plt.subplots(1,4)
aa = sns.countplot(df['RaceDesc'], ax=ax[0])
ab = sns.countplot(df['MaritalDesc'], ax=ax[1])
ac = sns.countplot(df['Sex'], ax=ax[2])
ad = sns.countplot(df['CitizenDesc'], ax=ax[3])

aa.set_xticklabels(aa.get_xticklabels(), rotation=90)
ab.set_xticklabels(ab.get_xticklabels(), rotation=90)
ad.set_xticklabels(ad.get_xticklabels(), rotation=90)

fig.show()

df[['MarriedID','GenderID','FromDiversityJobFairID']].describe()

### Objective
3. What are our best recruiting sources if we want to ensure a diverse organization?
<br>
3.1 Apart from white employee, majority of other races reccruited by Diversity Job Fair.

In [ ]:
pd.crosstab(df.RecruitmentSource, df.RaceDesc)

### Objective
4. Can we predict who is going to terminate and who isn't? What level of accuracy can we achieve on this?
<br>
4.1 Logistic regression shall be apply
<br>
4.2 Correlation matrix indicate that Diversity recruitement source and special project count has the highest association with temination followed by DaysLateLast30, marital status adn salary. 
4.3 However, based on the partial correlation results, both marital status and salary have low partial correlation with termination indicating that both could be not the reasons for employee left. 

In [ ]:
df_correl = df[['MaritalStatusID','GenderID','PerfScoreID','FromDiversityJobFairID','Salary','Termd','Department','RecruitmentSource','PerformanceScore','EngagementSurvey','EmpSatisfaction','SpecialProjectsCount','DaysLateLast30','Absences']]
sns.set(rc={'figure.figsize':(10,10)})
sns.set_context("talk", font_scale=0.7)

#sns = seaborn    
#corr = correlation
#annot = annotation (Label values - Correlation)

sns.heatmap(df_correl.corr(), cmap='Reds', annot=True)

In [ ]:
df_correl.pcorr().round(3)

### Objective
4. Can we predict who is going to terminate and who isn't? What level of accuracy can we achieve on this?
<br>
4.1.1 Most of the employee who are terminated mainly came from the special project count with none or more than 4.
<br>
4.1.2 Recruited from FromDiversityJobFair has higher change to be terminate.
<br>
4.1.3 Large portion of terminated employee come from production department

In [ ]:
fig, ax = plt.subplots(1,2)
aa = sns.countplot(x="FromDiversityJobFairID", hue="Termd", data=df, ax=ax[0])
ab = sns.countplot(x="SpecialProjectsCount", hue="Termd", data=df, ax=ax[1])
fig.show()

In [ ]:
ax = sns.countplot(x="Department", hue="Termd", data=df )

loc, labels = plt.xticks()
ax.set_xticklabels(labels, rotation=90);

In [ ]:
df_output = df.groupby('Termd').mean().T
df_output

### Objective
4. Can we predict who is going to terminate and who isn't? What level of accuracy can we achieve on this?
<br>
4.2 Split department intoeach variable to obtain production variable.
<br>

In [ ]:
df_copy = df.copy()
df_copy.columns = df_copy.columns.str.replace(' ', '')
df_copy = pd.get_dummies(df_copy, columns=['Department'], prefix = ['Department'])

### Objective
4. Can we predict who is going to terminate and who isn't? What level of accuracy can we achieve on this?
<br>
4.3.1 From Diversity Job Fair and days late last 30 days attained the highest accuracy score among 1 attribute combination.
<br>
4.3.2 Highest accuracy of 69.5% is obtained through Diversity Job Fair, Production department & DaysLateLast30 while any addition could not offer any uplift toward the accuracy
<br>

In [ ]:
#sklearn -> scikit-learn
from sklearn.linear_model import LogisticRegression
columns_1_1 = ['DaysLateLast30']
columns_1_2 = ['FromDiversityJobFairID']
columns_1_3 = ['SpecialProjectsCount']
columns_1_4 = ['Department_Production       ']

# Establishing the base dataset
X_1_1 = df_copy[columns_1_2]
X_1_2 = df_copy[columns_1_2]
X_1_3 = df_copy[columns_1_3]
X_1_4 = df_copy[columns_1_4]
y = df_copy['Termd'] 

from sklearn.metrics import accuracy_score

logmodel = LogisticRegression()

logmodel.fit(X_1_1,y)
predictions = logmodel.predict(X_1_1)
AS_1_1 = accuracy_score(y,predictions)

logmodel.fit(X_1_2,y)
predictions = logmodel.predict(X_1_2)
AS_1_2 = accuracy_score(y,predictions)

logmodel.fit(X_1_3,y)
predictions = logmodel.predict(X_1_3)
AS_1_3 = accuracy_score(y,predictions)

logmodel.fit(X_1_4,y)
predictions = logmodel.predict(X_1_4)
AS_1_4 = accuracy_score(y,predictions)

print("DaysLateLast30 =",AS_1_2*100)
print("FromDiversityJobFairID =",AS_1_2*100)
print("SpecialProjectsCount =",AS_1_3*100)
print("Production =",AS_1_4*100)

In [ ]:
columns_2_1 = ['FromDiversityJobFairID','Department_Production       ']
columns_2_2 = ['FromDiversityJobFairID','DaysLateLast30']
columns_2_3 = ['FromDiversityJobFairID','SpecialProjectsCount']
columns_2_4 = ['DaysLateLast30','Department_Production       ']
columns_2_5 = ['DaysLateLast30','SpecialProjectsCount']
columns_2_6 = ['Department_Production       ','SpecialProjectsCount']

# Establishing the base dataset
X_2_1 = df_copy[columns_2_1]
X_2_2 = df_copy[columns_2_2]
X_2_3 = df_copy[columns_2_3]
X_2_4 = df_copy[columns_2_4]
X_2_5 = df_copy[columns_2_5]
X_2_6 = df_copy[columns_2_6]
y = df_copy['Termd'] 

from sklearn.metrics import accuracy_score

logmodel = LogisticRegression()
logmodel.fit(X_2_1,y)
predictions = logmodel.predict(X_2_1)
AS_2_1 = accuracy_score(y,predictions)

logmodel.fit(X_2_2,y)
predictions = logmodel.predict(X_2_2)
AS_2_2 = accuracy_score(y,predictions)

logmodel.fit(X_2_3,y)
predictions = logmodel.predict(X_2_3)
AS_2_3 = accuracy_score(y,predictions)

logmodel.fit(X_2_4,y)
predictions = logmodel.predict(X_2_4)
AS_2_4 = accuracy_score(y,predictions)

logmodel.fit(X_2_5,y)
predictions = logmodel.predict(X_2_5)
AS_2_5 = accuracy_score(y,predictions)

logmodel.fit(X_2_6,y)
predictions = logmodel.predict(X_2_6)
AS_2_6 = accuracy_score(y,predictions)

print("FromDiversityJobFairID & Production =",AS_2_1*100)
print("FromDiversityJobFairID & DaysLateLast30 =",AS_2_2*100)
print("FromDiversityJobFairID & SpecialProjectsCount =",AS_2_3*100)
print("DaysLateLast30 & Production =",AS_2_4*100)
print("DaysLateLast30 & SpecialProjectsCount =",AS_2_5*100)
print("SpecialProjectsCount & Production =",AS_2_6*100)

In [ ]:
columns_3_1 = ['FromDiversityJobFairID','Department_Production       ','SpecialProjectsCount']
columns_3_2 = ['FromDiversityJobFairID','Department_Production       ','DaysLateLast30']
columns_3_3 = ['FromDiversityJobFairID','DaysLateLast30','SpecialProjectsCount']
columns_3_4 = ['DaysLateLast30','Department_Production       ','SpecialProjectsCount']

# Establishing the base dataset
X_3_1 = df_copy[columns_3_1]
X_3_2 = df_copy[columns_3_2]
X_3_3 = df_copy[columns_3_3]
X_3_4 = df_copy[columns_3_4]
y = df_copy['Termd'] 

from sklearn.metrics import accuracy_score

logmodel = LogisticRegression()
logmodel.fit(X_3_1,y)
predictions = logmodel.predict(X_3_1)
AS_3_1 = accuracy_score(y,predictions)

logmodel = LogisticRegression()
logmodel.fit(X_3_2,y)
predictions = logmodel.predict(X_3_2)
AS_3_2 = accuracy_score(y,predictions)

logmodel = LogisticRegression()
logmodel.fit(X_3_3,y)
predictions = logmodel.predict(X_3_3)
AS_3_3 = accuracy_score(y,predictions)

logmodel = LogisticRegression()
logmodel.fit(X_3_4,y)
predictions = logmodel.predict(X_3_4)
AS_3_4 = accuracy_score(y,predictions)


print("FromDiversityJobFairID & Production & Speial Project Count=",AS_3_1*100)
print("FromDiversityJobFairID & Production & DaysLateLast30=",AS_3_2*100)
print("FromDiversityJobFairID & DaysLateLast30 & Speial Project Count=",AS_3_3*100)
print("DaysLateLast30 & Production & Speial Project Count=",AS_3_4*100)

In [ ]:
columns_4_1 = ['FromDiversityJobFairID','Department_Production       ','SpecialProjectsCount','DaysLateLast30']

# Establishing the base dataset
X_4_1 = df_copy[columns_4_1]
y = df_copy['Termd'] 

from sklearn.metrics import accuracy_score

logmodel = LogisticRegression()
logmodel.fit(X_4_1,y)
predictions = logmodel.predict(X_4_1)
AS_4_1 = accuracy_score(y,predictions)

print("FromDiversityJobFairID & Production & Speial Project Count & DaysLateLast30=",AS_4_1*100)

